# Word2Vec
在许多自然语言处理任务中，单词通常由他们的 [tf-idf]() 分值来表示。这种分值（只是一个标量scalar）只能表征出一个词在一个文档中的重要程度，但无法给出任何语义层面的解释，意思就是你只是一个标量数值，能够显示出单词的一种重要程度或者权重，但是却无法表达出单词的语义，含义往往和词的上下文以及词的近义词反义词等有关系。Word2Vec 是一种能够采用神经网络模型，它对无标签的语料库进行训练，最终得出每个单词的一个词向量（也就是说这个单词通过向量vector来表征），这种词向量可以理解成是对单词在相关语料库所表现出来的隐含语义的一种编码。词向量表示方式是非常有用，主要体现在一下两点：
 1. 我们能通过计算余弦相似度来度量两个单词的语义相似度。所有单词已经在同一个高维度空间，在这个维度空间，我们可以采用各种数学定义上的距离度量，来度量两个单词的语义相似度，如欧式距离、马氏距离、余弦距离、汉明距离、曼哈顿距离、皮尔逊相关系数。
 2. 我们可以使用词向量作为特征向量做各种有监督NLP机器学习任务（有机器学习基础的话就知道，所有机器学习模型的输入最后都变成了一个多维特征向量），比如文档分类，命名实体标注，文本情感分析。在词向量中隐含的语义信息能够作为机器学习输入数据的强有力的特征，传统机器学习的输入数据的每个维度特征，有很大一部分是人为设计的，不是靠机器单纯学出来的，比如某个维度表示词的频率，词的词性（adj,adv,noun.）,词的表意（good or bad or neutral），词的长度，词的音节，音素等等，需要大量的语言学家设计出这些特征并做提取。
 
你可能会问“我们怎么才能高效的学习到单词的语义呢？”. 确实，目前为止还无法科学解释为什么能够学出来，因为使用了神经网络炼金术-:)，但是试验效果出奇的好，而且直觉上来讲，近义词(synonyms)在空间上距离相近，反义词(antonyms)在空间上距离远。更惊喜的是，词向量还能够进行类比推理，这个是论文中的经典例子“Woman is to queen as man is to king”. 结果在训练出来的词向量里面可以直接得出以下公式：
$$
v_{queen} - v_{woman} + v_{man} \approx v_{king}
$$
这里 $v_{queen}$, $v_{woman}$, $v_{man}$ 和 $v_{king}$ 分别表示各自词的词向量。这些实验结果强烈的暗示了词向量技术是对他们代表的单词在某个隐含语义空间的编码。

下面我来讲解Word2Vec常用的两个模型算法，skip-gram 和 continuous bag-of-words（CBOW） 模型。这两种模型都是单隐层的神经网络模型，并通过神经网络的经典学习方法反向传播(backpropagation)和随机梯度下降(stochastic gradient descent)进行学习。
 - skip-gram 是一个根据一个输入词预测他上下文单词的模型
 - cbow 是给定上下文单词，预测中心单词的模型

## The Skip-Gram Model
我们需要使用数学符号来定义和描述我们的模型以及输入。skip-gram 的输入是单个单词 $w_I$ 输出是这个单词的上下文 $\{ w_{O,1},...,w_{O,C} \}$, 其中 $C$ 表示的是单词上下文的窗口大小，就是以该单词为中心，左右取多少个上下文单词。比如，"I drove my car to the store", 如果输入单词取 car, $C=3$,那么我们的输出上下文就是 {"I", "drove", "my", "to", "the", "store"}, 所有这些单词都是通过 one-hot 编码的，他们都是长度为$V$(表示训练所使用的词汇表的长度)，相应单词索引位置值为1，其他位置值为0的向量。这些训练数据都来自纯文本，我们有无穷无尽的文本可以拿来训练。

接下来我们就来定义模型：
$X$代表输入单词 $w_I$ 的one-hot编码输入，$\{y_1,...,y_C\}$ 代表输出上下文的one-hot编码，每个都是one-hot编码。$V \times N$ 矩阵 $W$ 是输入层和隐藏层之间的全连接矩阵，$W$第 i 行就代表与其对应的第 i 个单词的词向量。你可以试试用 one-hot 编码输入$X$和该矩阵做点乘 $W^TX$ 就明白了。隐层和输出层之间也有一个 $N \times V$ 矩阵 $W^{'}$, 中间的隐层含有 $N$ 个节点，也是最后词向量的长度大小。

![skip-gram-arch](skip-gram-arch.png)


输入层到隐藏层计算方式：
$$
\mathbf{h} = W^T \cdot X = v_{w_I}^T
$$
输出层是C个单词，属于C个多项分布$\{y_1,...,y_C\}$, 针对此的概率公式是：
$$
P(w_{c,j}|w_I) = y_{c,j} = \frac{\exp (u_{c,j})}{\sum_{k=1}^{V} \exp(u_{c,k})}
$$
这里的关键就是，我们的损失函数和常见的softmax多分类交叉熵稍微有点不同，因为这里又多增加了一维参数C,代表C个输出上下文单词的分布，他们在我们的模型考虑中相互独立，实际上并不是独立的，但是为了简化，我们就认为他们是独立的。这些输出单词共享权重$W'$, $u_{c,j} = u_{j} ={v^{'}_{w_j}}^T \cdot \mathbf{h}$, $v^{'}_{w_j}$ 代表 $W'$的第j列，他也是一种词向量，但是这个词向量意义和矩阵$W$ 代表的意义不同。

最后我们尝试推一下最后的交叉熵损失，论文直接给出了一个很突兀的公式，其实这个公式应该是下面这个公式化简出来的。


关于模型公式和求导反向传播详细推导，可以参考论文[word2vec Parameter Learning Explained](http://wiki.hacksmeta.com/static/pdf/word2vec-Parameter-Learning-Explained-5.pdf)，求导非常容易出错而且对于矩阵求导，维度很容易搞错，还好有TensorFlow等工具可以自动差分求导，我们就不用管求导了。


skip-gram 是一个根据一个输入词预测他上下文单词的模型，第一个矩阵目的是要将one-hot 编码的输入词，映射到词向量空间，也就是说第一个矩阵学习的是词向量空间。其后，第二个矩阵的目的是将词向量再映射回one-hot编码（但并不是原来输入词的，而是输入词上下文的），但是这个 $W'$ 不是简单的 $W^T$, 而是新引入一个矩阵，他学习的其实是输入单词向量和其上下文单词向量之间的协同关系，而不是学习的词向量，这也是我们使用第一个矩阵作为最后学习到的词向量的原因。

## 几个问题
### 为什么Word2Vec 不使用正则项？
因为Word2Vec的目的并不是为了让这个模型去适应语料库意外的语料，也不是用来再去预测其他的未见过测用例，他只是为了训练出当前语料库中所有单词的词向量。只需要拟合当前数据即可，不需要泛化。
### 为什么Word2Vec 隐藏层没有使用激活函数？
目的不是为了泛化，也不需要激活，激活是为了

### Word2Vec 两个矩阵能够使用同一个吗？也就是说 W.T = W'
目前网上没人思考过这个问题，如果我们得到了第一个矩阵是词嵌入矩阵，他的每一行代表的是对应单词的词向量，然后采用计算隐层向量（他其实就是某个词向量）和其他所有词向量的距离，用这个距离来度量输入词和其他词之间的关系，以此再进行softmax概率分布来评估误差，但是你不能还是用第一个矩阵来学习，因为第一个矩阵是用来学习词嵌入的，第二个矩阵是用来学习词嵌入和他的上下文单词关系的，这是两个不同的学习对象，因此我们需要再引入一个新矩阵来学习。

目前我认为如果不能的话，应该是这个原因。但是如果可以的话，我觉得也说得通，这得做实验了。

### 为什么取第一个矩阵做词向量而不是第二个矩阵？
目前网上没人思考过这个问题，目前的解释认为从one-hot 输入到隐层其实是编码进入词向量空间，第一层映射是真实的拿到词向量，也就是说第一个矩阵才是学习我们需要的词向量的矩阵，从隐层到输出是解码到one-hot，但是第二层映射解码出来的目的不是变会原来输入单词的one-hot,而是尽可能匹配出上下文单词的one-hot。也即是说第二层映射过程中发生的是距离计算和度量，第二个矩阵学习到的是某个词向量和他上下文距离远近的关系。

## Continous bag of words model
cbow 是给定上下文单词，预测中心单词的模型，和skip-gram刚好是相反的。
![cbow](cbow-arch.png)


这里关键一点是，我们现在的输入变成多个向量了，如何得到一个隐层向量呢？论文中简单粗暴的用了平均方法：

\begin{align*}
\mathbf{h} &= \frac{1}{C} W^T(x_{1} +x_2+...+x_{C}) \\ &= \frac{1}{C}(v_{w_1} + v_{w_2}+...+v_{w_C})^T
\end{align*}
然后，从隐层到输出层，直接使用矩阵 $W'$ 相乘，这样可以得到线性和之后进行softmax操作：

$$
\mathbf{u} = W^{'T} \cdot h
$$
然后我们计算softmax概率公式：
$$
P(w_j|w_I) = y_j = \frac{\exp(u_j)}{\sum\limits_{k\in V} \exp(u_k)} =   \frac
{\exp(v_{w_j}^{'T}\cdot v_{w_I})}{\sum\limits_{k\in V} \exp(v_{w_k}^{'T}\cdot v_{w_I})}
$$
最后我们采用交叉熵计算损失：


## 优化
当拥有10000个单词的词汇表，我们如果想嵌入300维的词向量，那么我们的输入-隐层权重矩阵和隐层-输出层的权重矩阵都会有 10000 x 300 = 300万个权重，在如此庞大的神经网络中进行梯度下降是相当慢的,模型参数和数据量都是百万级亿级别，模型和数据量都太大了。而且容易学到一些不重要的词汇。

### 抽样率和负采样（Negative Sampling）
Word2Vec的作者在它的第二篇论文中强调了这些问题，下面是作者在第二篇论文中的三个创新：
 - 将常见的单词组合（word pairs）或者词组作为单个“words”来处理。
 - 对高频次单词进行抽样来减少训练样本的个数。
 - 对优化目标采用“negative sampling”方法，这样每个训练样本的训练只会更新一小部分的模型权重，从而降低计算负担。
#### 词组做成一个单词
就是把一些常见的词组当成一个单词来看待，而不是分开看待，比如“New York” 可以不拆开，他是一个地名。这样学习到的语义更加有意义。
#### 高频抽样
文档中有些词可能出现频率非常高，英文中单词"the" 的概率几乎非常大，中文的"的"字出现概率也很大，他们提供的信息量并不强，过多的加入训练反而会影响训练结果。
![negative-sample](negative-sample.jpg)

我们在采样的生成输入数据的过程中可以以某个概率保留这个词，词频越大的词，确定性越强，往往说明信息含量低，即信息熵比较低，保留的概率越低，我们可以定义保留概率
$$
P(w_i) = (\sqrt \frac{f(w_i)}{0.001} + 1) \times \frac{0.001}{f(w_i)}
$$
其中，$f(w_i)$ 表示的是一个词的词频的函数，一般我们取得就是词频率。
#### 负采样
word2vec里面一个重点都是one-hot 编码导致的词汇表维度非常大，而大的词汇表又会导致我们的参数矩阵非常大，再加上训练语料都是数亿级别，训练非常慢。我们模型训练过程中拟合one-hot编码有一个特点，就是我们只是希望输出单词上对应索引位置的值接近最大，其他地方都是0，所以我们可以选择只更新哪些高频词汇影响的部分权值矩阵。

比如，当我们用训练样本 ( input word: "fox"，output word: "quick") 来训练我们的神经网络时，“ fox”和“quick”都是经过one-hot编码的。如果我们的vocabulary大小为10000时，在输出层，我们期望对应“quick”单词的那个神经元结点输出1，其余9999个都应该输出0。在这里，这9999个我们期望输出为0的神经元结点所对应的单词我们称为“negative” word。

为了加快训练速度，我们可以采样negative word，采样根据词频来采，这次是词频越高，被采样的概率越大。代码中采样负单词的采样概率公式如下：
$$
P(w_i) = \frac{f(w_i)^{0.75}}{\sum_{j=0}^n(f(w_i)^{0.75})}
$$

### 层次softmax(Hierarchical Softmax)
首先我们根据语料库中单词的词频，采用Huffman树将单词进行Huffman编码，构造出一颗Huffman树，Huffman树的每个叶子节点都是词汇表中的单词，中间节点都是一个小二分类器，采用sigmoid激活函数进行二分。
![huffman-tree](huffman-tree.png)

在输出层我们不在直接使用softmax对所有的词汇计算加权和，这会需要$N \times N  \times V$计算量，这里往往 $V \gg N$。而采用层次二分法，对输出的隐层向量h，我们从Huffman树的根节点开始，做二分模型，沿着内部节点一直走到叶子节点，这个时候我们的计算量是$N \times N \times \log V$

$$
P(+) = \sigma(x_w^T\theta) = \frac{1}{1+e^{-x_w^T\theta}}
$$

![word2vec-hierarchical-softmax](word2vec-hierarchical-softmax.png)

# 参考
 - [Word2vec数学原理全家桶](http://shomy.top/2017/07/28/word2vec-all/)
 - [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
 - [Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
 - [基于Hierarchical Softmax的模型](http://www.cnblogs.com/pinard/p/7243513.html)
 - [An Intuitive Understanding of Word Embeddings: From Count Vectors to Word2Vec](https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/)

# 基于TensorFlow实现Skip-Gram

In [75]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import random
import jieba
import numpy as np
from six.moves import xrange
import tensorflow as tf
import datetime as dt

In [76]:
import jieba
import re

def get_stop_words(filename='./data/stopwords.txt'):
    stop_words = []
    with open(filename, 'r', encoding='utf-8') as f:
        stopwords = [line[:-1] for line in f]
    stop_words = set(stop_words)
    print("获取{n}个停用词".format(n=len(stop_words)))
    return stop_words

def get_sentences(content):
    content = content.decode("utf8")
    eng_tokens = ['!', ',', '.', '?', ';', ':', ' ']
    zh_tokens = ['！', '，', '。', '？', '；', '：', ' ']
    zh_tokens = [token.decode('utf8') for token in zh_tokens]
    sentences = []
    line = ""
    for ch in content:
        line += ch
        if ch in eng_tokens or ch in zh_tokens:
            if len(line) > 0:
                sentences.append(line.replace(ch, "").encode("utf8"))
            line = ""
    if len(line) > 0:
        sentences.append(line.encode("utf8"))
        
    return sentences

def get_raw_sentence(sentence):
    puncts = "[\s+\.\!\/_,$%^*()<>+\"\'\”\“]+|[+——！，。？、~@#￥%……&*（）《》]+"
    text = re.sub(puncts.decode("utf8"), "".decode("utf8"), sentence.decode("utf8"))
    return text
    
def get_words_list(filename="./data/doupocangqiong.txt"):
    words_list = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.replace('\n', '').replace(' ', '')
            if re.match(r'^https?:/{2}\w.+$', line) or re.match(r'^ftp:/{2}\w.+$', line):
                continue
            # 先分句
            sentences = get_sentences(line)
            for sentence in sentences:
                # 去除句子中的特殊标点符号
                sentence = get_raw_sentence(sentence)
                words = list(jieba.cut(sentence, cut_all=False)) if len(sentence) > 0 else []
                words_list.extend(words)
    return words_list        

In [ ]:
words = get_words_list()

In [ ]:
for word in words[:10]:
    print(word)

In [ ]:
def build_dataset(words, n_words):
    counter = [['UNK', -1]]
    counter.extend(collections.Counter(words).most_common(n_words - 1))
    word2index = {}
    for word, _ in counter:
        word2index[word] = len(word2index)
    word_num_data = []
    unk_count = 0
    for word in words:
        if word in word2index:
            index = word2index[word]
        else:
            index = 0
            unk_count += 1
        word_num_data.append(index)
    counter[0][1] = unk_count
    index2word = dict(zip(word2index.values(), word2index.keys()))
    return word_num_data, counter, word2index, index2word
    
def collect_data(vocabulary_size = 10000):
    words = get_words_list()
    data, count, word2index, index2word = build_dataset(words, vocabulary_size)
    del words
    return data, count, word2index, index2word

对于输入的文档单词序列表，需要处理成数字序列才能训练，上述就是拿到词汇表的过程，`word2index` 就是根据词查其索引的词汇表，`index2word` 是根据词的索引查词，`word_num_data` 是训练语料，但是是词对应的索引号构成的序列。`counter` 就是词频表。

In [ ]:
data_index = 0
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [skip_window centor_word skip_window]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window] # this is the input word
            context[i * num_skips + j, 0] = buffer[target] # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index+1)%len(data)
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

比如在 5-gram 中，"the cat sat on the", 假设输入词是 "sat"， 那么上下文单词会从`['the', 'cat', 'on', 'the']` 随机选取。选取个数由参数 `num_skips` 来确定. `skip_window` 就是窗口大小，这里 5-gram 就是 2. `span` 看公式就能知道是什么意思了，就是整个一次输入和上线文总总长度。

这里在采样生成训练数据的时候，我们采用的是滑动窗口，一个队列就可以完成这个算法了。整个数据流data的全局索引由全局变量 `data_index` 来控制。

注意训练skip-gram 的时候，往往和论文中那个模型示例图不一样的地方是，不是输入一个词，并不会同时输出所有的上下文单词，而是只选择一个输出，这可以简化训练过程，而且不影响训练结果，因为我们认为这是独立事件。

train/test/validate


In [ ]:
# 验证集
# 从0-100 选取16个整数，对一个的是100个最高频词的索引，用这些词来评估模型学习进度
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64

vocabulary_size = 5000
data, counter, word2index, index2word = collect_data(vocabulary_size)

batch_size = 128
embedding_size = 128 # 词向量长度
skip_window = 1
num_skips = 2

In [ ]:
graph = tf.Graph()
with graph.as_default():
    
    # 设置模型输入输出和验证集 placeholder
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # 设置embeddings tensor, input and hidden layer
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)) # shape = (V, N)
    embed = tf.nn.embedding_lookup(embeddings, train_inputs) # shape (N, )

    # hidden layer and output
    weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], 
                                              stddev=1.0/math.sqrt(embedding_size))) # shape (V, N)
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases

    # loss
    train_one_hot = tf.one_hot(train_context, vocabulary_size)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out,
                                    labels=train_one_hot))
    # algorithm sgd
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

    # 使用余弦距离评价验证集和对应词向量之间的相似度
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)


    init = tf.global_variables_initializer()

$$
similarity = cos(\theta) = \frac{\textbf{A}\cdot\textbf{B}}{\parallel\textbf{A}\parallel_2 \parallel \textbf{B} \parallel_2}
$$


In [ ]:
def run(graph, num_steps):
    with tf.Session(graph=graph) as session:
        init.run()
        print("Initialized")
        
        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}
            
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val
            
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                print("Average loss at step ", step, ': ', average_loss)
                average_loss = 0
                
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = index2word[valid_examples[i]]
                    top_k = 8
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s: ' % valid_word
                    for k in range(top_k):
                        close_word = index2word[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
            final_embeddings = normalized_embeddings.eval()


In [ ]:
num_steps = 100000
softmax_start_time = dt.datetime.now()
run(graph, num_steps=num_steps)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))

with graph.as_default():

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()

num_steps = 50000
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} minutes to run 100 iterations".format((nce_end_time-nce_start_time).total_seconds()))


In [ ]:
# Step 6: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='images/tsne3.png',fonts=None):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                    fontproperties=fonts,
                    xy=(x, y),
                    xytext=(5, 2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')

    plt.savefig(filename,dpi=800)

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    
    #为了在图片上能显示出中文
    font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)
    
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels,fonts=font)
    
    
except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")